<a href="https://colab.research.google.com/github/atulloona4/Deeplearningjupyter/blob/master/pyspark_with_rdd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install -U sentence_transformers

Mounted at /content/drive
     |████████████████████████████████| 85 kB 2.2 MB/s 
     |████████████████████████████████| 2.9 MB 24.1 MB/s 
     |████████████████████████████████| 1.2 MB 29.8 MB/s 
     |████████████████████████████████| 52 kB 1.3 MB/s 
     |████████████████████████████████| 895 kB 72.3 MB/s 
     |████████████████████████████████| 3.3 MB 30.2 MB/s 
     |████████████████████████████████| 636 kB 32.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126710 sha256=661739944a93dcc8e402d159357565245476ba04b19fb77459be7cd7fac5234c
  Stored in directory: /root/.cache/pip/wheels/d1/c1/0f/faafd427f705c4b012274ba60d9a91d75830306811e1355293
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!cp /content/drive/MyDrive/spark/spark-3.1.2-bin-hadoop3.2.tar . 

In [ ]:
!tar xf spark-3.1.2-bin-hadoop3.2.tar

In [ ]:
!pip install -q findspark


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.1.2-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()

In [ ]:
findspark.find()

'spark-3.1.2-bin-hadoop3.2'

In [ ]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local") \
                    .appName('sbert_encode') \
                    .getOrCreate()
sc = spark.sparkContext

spark.conf.set("spark.sql.execution.arrow.enabled", "true")
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "64")
from pyspark.sql.functions import col, pandas_udf
import pyspark.sql.functions as f
from pyspark.sql.types import *
import pandas as pd


In [ ]:
spark

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [ ]:
def encode(para):
  emb = model.encode(str(para))
  return emb

In [ ]:
data = spark.read.text("/content/drive/MyDrive/spark/*.txt").rdd
embeddings = data.map(encode)

In [ ]:
embeddings.collect()

[array([ 3.32888961e-02, -2.75888089e-02, -6.30664900e-02,  1.96113922e-02,
         4.24116990e-03,  1.55491075e-02, -4.36099945e-03,  2.49080751e-02,
        -5.74237444e-02,  2.71100178e-02,  3.60431783e-02,  3.06230783e-02,
         7.71098072e-03, -3.16267870e-02, -1.23338945e-01,  3.35043333e-02,
         6.49990398e-05,  5.73107833e-03,  1.87259016e-03,  1.30795553e-01,
        -2.06740443e-02,  4.93193278e-03, -2.17380635e-02, -4.66456375e-04,
         6.61992049e-03,  4.32886556e-02,  4.24851291e-02, -1.38143182e-03,
         9.00232140e-03,  9.29042418e-03, -3.55093479e-02,  2.87416168e-02,
        -4.16647978e-02, -3.63242114e-03,  1.83828585e-02, -2.99956789e-03,
         3.31720449e-02, -4.35135923e-02, -3.35485190e-02, -8.11090786e-03,
         4.44432423e-02, -1.60513874e-02,  5.94184361e-03,  7.57090524e-02,
         5.04237637e-02, -8.00244212e-02, -2.75370311e-02,  6.51860014e-02,
         4.05388325e-02,  8.59153941e-02, -1.05268592e-02,  3.33621725e-02,
        -2.8

In [ ]:
mkdir /content/drive/MyDrive/spark/news_data

mkdir: cannot create directory ‘/content/drive/MyDrive/spark/news_data’: File exists
